<a href="https://colab.research.google.com/github/envgp/taking_the_pulse_of_the_planet/blob/main/notebooks/pulse_assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1: Taking the Pulse of the Atmosphere

Seogi Kang (sgkang09@stanford.edu)& Rosemary Knight(rknight@stanford.edu), Stanford Environmental Geophysics Research Group

This assignment is due on`2023-1-24`. Please put your answers within this notebook and share the completed notebook with the graders, bsalvado@stanford.edu and sgkang09@stanford.edu, using the Share banner located at the top right corner of this notebook. When sharing your notebook, please change the name of the notebook and add your name and sunetid (e.g., `pulse_course_assignment_1_firstname_lastname_sunetid.ipynb`)

## INTRODUCTION TO THE ASSIGNMENT

In this assignment, we will explore the changing concentration of greenhouse gases in the atmosphere -  CO2 and CH4, which is what is driving climate change. We will also review measurement of the other trace gases NO2 and CO which are not primary drivers of climate change, but do have an impact on air quality, a concern in terms of sustainability with related environmental justice issues. 

## DATA SETS

Data sets used this assignment are: 
- CO2 - one time-series at one location
- Gridded data - CO2 in space and time (time-series at many locations)
- Gridded data - CH4 in space and time (time-series at many locations)
- Gridded data - NO2 in space and time (time-series at many locations)
- Gridded data - CO in space and time (time-series at many locations)

## BUILDING YOUR TOOLBOX

This is the first assignment of the course, so we will use this and the next assignment to introduce core tools including Google Colab (similar to Jupyter Notebook) and Python packages that we are going to use throughout the course. The list of core Python packages are 

1. `numpy`: for calculating mathematical functions
2. `matplotlib`: for visualization
3. `pandas`: for handling data in a table form
4. `xarray`: for handling gridded data 
5. `cartopy`: for visualizing gridded data as a 2D map
6. `ipywidgets`: for creating interactive apps(exploration of data)
7. `geopandas`: for handling geographic data (e.g., country boundaries)


## THE LEARNING GOALS FOR THE WEEK 
(where the course learning goals are in plain text, and the focus this week is in *italics*)



*   learn about the ways in which climate change and human activity are impacting planet Earth, *with a focus this week in observing – in the data - increases in gases that are driving climate change and negatively impacting air quality; and exploring the link to emissions.*
*   become familiar with the wide range of sensors available to study various components of the Earth system. These include sensors on satellites, aircraft, ground-based platforms, and deployed above or beneath the surface on land or water. *This week we will work with data acquired using one ground-based measurement station and satellite systems.*
*   become familiar with the basic physical principles (resolution, sampling, processing workflows, etc.) common to all sensors, *working this week with a range of data sets.*
*   work with various sources of data, learning how to access, analyze, synthesize, and describe the data to quantify trends; think critically and creatively about how to project these trends into the future. *This week we will work with a number of tools, which will be used again in this course, to transform or analyze the data in various ways and then visualize it.*
*   describe the complex interactions between human activity and various components of the Earth system. This week a key question: *what is the link between the spatial pattern in emissions and the observed changes in the concentrations of greenhouse and other gases in the atmosphere*. 
*   become motivated to think about new sensors and new ways of using sensor data to study the planet. *This is always the last question in each assignment. Given all that you now know about greenhouse gases, emissions, climate change, what does the planet and all forms of life need you to design and deploy?*




## STARTING THE ASSIGNMENT - A WALK THROUGH SOME OF THE DATA AND TOOLS

## Install required packages

In [ ]:
!pip install xarray numpy pandas geopandas cartopy==0.19.0.post1 ipywidgets

In [ ]:
!pip uninstall -y shapely

In [ ]:
!pip install shapely --no-binary shapely

## Download required data

In [ ]:
!git clone https://premonition.stanford.edu/sgkang09/taking_the_pulse_atmosphere_data.git

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Working with CO2 data - one time-series at one location in a table format using `pandas`

Let's start with a monthly time-series of CO2 level measured at Mauna Loa Observatory. We are going to read the CO2 level in a csv format using `pandas`. Loaded table is the pandas' `dataframe`, which allows you to explore the data. 

In [ ]:
import pandas as pd

In [ ]:
df_co2 = pd.read_csv('./taking_the_pulse_atmosphere_data/co2_gml.csv')

In [ ]:
df_co2.head(2)

In [ ]:
df_co2.tail(3)

Access data in pandas dataframe.

In [ ]:
import datetime
# generate DatetimeIndex from years and months
years = df_co2['year'].values.astype(int)
months = df_co2['month'].values.astype(int)
times = [datetime.date(years[ii], months[ii], 1) for ii in range(df_co2.shape[0])]
time_series = pd.to_datetime(times)
df_co2['time'] = time_series
df_co2 = df_co2.set_index('time')

Explain the above cell line-by-line

- numpy array
- list comprehension
- `datetime.date`
- `pd.to_datetime`
- ...

> **_NOTE:_**  useful keyboard shortcuts:
- running a cell: `shift + enter`
- auto completion: `control + space` on windows or `command + space` on mac



Now we are ready to plot the data in the CO2 time-series. Let's plot them using `pandas`.

In [ ]:
df_co2.plot(y='monthly_average_co2')

We could fine-tune the plot using `matplotlib`.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10, 5))
df_co2.plot(y='monthly_average_co2', ax=ax)
df_co2.plot(y='de-seasonalized_co2', ax=ax)
ax.grid(True, alpha=0.2)
ax.set_ylabel("CO2 level (ppm)")
ax.legend()

Suppose you just wanted to plot the data between 1960 and 1980, you could use `numpy`. 

In [ ]:
import numpy as np

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10, 5))
year_min = 2015
year_max = 2022
# find indicies between 1960 and 1980
index_time = np.logical_and(year_min<time_series.year, year_max>=time_series.year)
df_co2.loc[time_series[index_time]].plot(y='monthly_average_co2', ax=ax)
df_co2.loc[time_series[index_time]].plot(y='de-seasonalized_co2', ax=ax)
ax.grid(True, alpha=0.2)
ax.set_ylabel("CO2 level (ppm)")
ax.legend()

Although the table includes `de-seasonalized_co2`, as an exercise we are going to do this ourselves, starting with the total data and then removing seasonal changes by using a moving average.

In [ ]:
moving_average_window_size = 12
df_series = df_co2['monthly_average_co2'].rolling(window=moving_average_window_size, center=True).mean()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10, 5))
year_min = 1960
year_max = 1980
index_time = np.logical_and(year_min<time_series.year, year_max>=time_series.year)
ax.plot(time_series[index_time], df_co2['monthly_average_co2'].values[index_time], label="CO2", marker='None')
ax.plot(time_series[index_time], df_co2['de-seasonalized_co2'].values[index_time], label="CO2 without seasonal changes", marker='None')
df_series.loc[time_series[index_time]].plot(label='C02 moving average')
ax.grid(True, alpha=0.2)
ax.set_ylabel("CO2 level (ppm)")

ax.legend()

You could also estimate a linear trend using linear regression. 

In [ ]:
def linear_fit(time_series, data):
    # assume the time series has a regular sampling rate
    times = np.arange(len(time_series))
    G = np.c_[times, np.ones_like(times)]
    Pinv = np.linalg.pinv(G)
    m = Pinv.dot(data)
    predicted_data = G @ m
    slope = m[0,:]
    intercept = m[1,:]
    return predicted_data, slope, intercept

In [ ]:
predicted_data, slope, intercept = linear_fit(time_series[index_time], df_co2['monthly_average_co2'].values[index_time].reshape([-1,1]))

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10, 5))
ax.plot(time_series[index_time], df_co2['monthly_average_co2'].values[index_time], label="CO2", marker='None')
ax.plot(time_series[index_time], predicted_data, label="Linear fit", marker='None')
ax.grid(True, alpha=0.2)
ax.set_ylabel("CO2 level (ppm)")
ax.legend()

Let's add the linear trend back to the dataframe.

In [ ]:
# Estimate the linear trend for all times
predicted_data, _, _ = linear_fit(time_series[:], df_co2['monthly_average_co2'].values[:].reshape([-1,1]))
df_co2['linear_trend_co2'] = predicted_data

In [ ]:
df_co2.head(3)

The data are monthly, but sometimes we are interested in looking at annual trends, so summing all the months in each year. We are going to use the `groupby` operation available through the pandas dataframe.

In [ ]:
df_co2_annual = df_co2.groupby('year')['monthly_average_co2'].mean()

In [ ]:
df_co2_annual.head(3)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10, 5))
df_co2_annual.plot(legend=False, ax=ax)
ax.grid(True, alpha=0.2)
ax.set_ylabel("CO2 level (ppm)")

## Working with gridded CO2 data using `xarray` and `cartopy`

### Introducing `xarray`

Now, we are going to load the gridded CO2 data in a [NETCDF](https://www.unidata.ucar.edu/software/netcdf/) format - a widely used data format for various atmopheric data. 

In [ ]:
import xarray as xr

In [ ]:
co2_data = xr.load_dataset('./taking_the_pulse_atmosphere_data/co2.nc')

Provide a nice summary of data structure. 

In [ ]:
co2_data

Let's get some information about the data. 

- Time range
- Temporal resolution 
- ... 

In [ ]:
co2_data.time.min()

In [ ]:
co2_data.time.max()

In [ ]:
co2_data.lon

How do you access the data?

In [ ]:
# demo - use co2 

Can we plot select a time slice of the data and plot a 2D map using `xarray`?

In [ ]:
i_time = 0
co2_data.co2.isel(time=i_time).plot(add_colorbar=False, cmap='hot_r')

Or you could also input a date. 

In [ ]:
time = '2016-1-10'
co2_data.co2.sel(time=time, method='nearest').plot(add_colorbar=False, cmap='hot_r')

### Introducing cartopy

2D maps are very colorful, but it would be helpful if we could see the continental boundaries.

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cf

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.axes.add_feature(cf.COASTLINE)

time = '2016-1-10'
out = co2_data.co2.sel(time=time, method='nearest').plot(add_colorbar=False, cmap='hot_r', ax=ax)
cb = plt.colorbar(out, fraction=0.04, orientation='horizontal')
cb.set_label("Monthly rate (ppmv/month)")
ax.set_aspect(1)

gl = ax.gridlines(
    crs=ccrs.PlateCarree(), 
    draw_labels=True,
    linewidth=1, 
    color='gray', 
    alpha=0.5, 
    linestyle='--'
)

We could also select multiple times. Let's say we want all data between 2020-01 and 2021-12. Then we could first find corresponding indices of `time_series_co2` then input them to the `co2_data`.

In [ ]:
time_series_co2 = pd.to_datetime(co2_data.time.data)
index_tmp = np.logical_and(time_series_co2.year>=2015, time_series_co2.year<=2019)
co2_data_tmp = co2_data.sel(time=time_series_co2[index_tmp])
co2_data_tmp

Access all the CO2 data and plot them in a histogram. 

In [ ]:
# comment about the Mauna Loa data. 
out = plt.hist(co2_data.co2.data.flatten(), bins=30, edgecolor='k')
plt.xlabel("CO2 level (ppm)")

Now let's make an interactive plot. 

In [ ]:
from ipywidgets import widgets, interact
def plot_co2(i_time):
    fig = plt.figure(figsize=(20, 10))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.axes.add_feature(cf.COASTLINE)

    time = '2016-1-10'
    out = co2_data.co2.isel(time=i_time).plot(
        add_colorbar=False, cmap='hot_r', ax=ax,
        # fix the color range
        vmin=390, vmax=420
      )
    cb = plt.colorbar(out, fraction=0.04, orientation='horizontal')
    cb.set_label("Monthly rate (ppmv/month)")
    ax.set_aspect(1)
    ax.set_aspect(1)
    plt.show()

Before you run the cell below, and toggle the time slider through the data, first think about what you expect to see. The best way to catch problems with data or with your analysis is to first THINK about what you expect to see in the data. If what you see is very different from what you expected to see, you might want to check on the data and/or your workflow. 

In [ ]:
interact(plot_co2, i_time = widgets.IntSlider(min=0, max=83, step=12))

We can do many more things using `xarray`. Let's calculate the mean CO2 values along the axes of Latitude and Longitude. Then plot the resulting time-series.

In [ ]:
co2_mean_time_series = co2_data.mean(dim=['lat','lon'])
# below does not work..., unfortunately
# co2_mean_time_series.plot()
plt.plot(co2_mean_time_series.time, co2_mean_time_series.co2.data)

Or we could calculate the mean along the longitudinal axis (which allows us to see what is usually the greater variation in the latitudinal direction), and plot profiles of CO2 at different times.

In [ ]:
co2_lat = co2_data.mean(dim=['lon'])
# use the same date
out = co2_lat['co2'].isel(time=[0, 2, 4, 6, 8]).plot.line(y='lat', hue='time', add_legend=True)

Now we are going to calculate the rate of change in the monthly CO2 measurements at every grid point in space. What we display is called a trend map.

In [ ]:
# Get co2 data as a numpy array, then reshape
co2_vec = co2_data.co2.data.reshape((co2_data.co2.data.shape[0], np.prod(co2_data.co2.data.shape[1:])))
time_series_co2 = pd.to_datetime(co2_data.time.data)
co2_linear_fit, slope, intercept = linear_fit(time_series_co2, co2_vec)

# Add resutls back to co2_data
co2_data['slope'] = (("lat", "lon"), slope.reshape(co2_data.co2.data.shape[1:]))
co2_data['intercept'] = (("lat", "lon"), intercept.reshape(co2_data.co2.data.shape[1:]))
co2_data['co2_linear_fit'] = (("time", "lat", "lon"), co2_linear_fit.reshape(co2_data.co2.data.shape))

time_min = pd.to_datetime(time_series_co2).min().strftime('%Y-%m')
time_max = pd.to_datetime(time_series_co2).max().strftime('%Y-%m')

Then plot the trend map. 

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.axes.add_feature(cf.COASTLINE)

out = co2_data['slope'].plot(ax=ax, add_colorbar=False, cmap='hot_r')
cb = plt.colorbar(out, fraction=0.04, orientation='horizontal')
cb.set_label("Monthly rate (ppmv/month)")
ax.set_aspect(1)
gl = ax.gridlines(
    crs=ccrs.PlateCarree(), 
    draw_labels=True,
    linewidth=1, 
    color='gray', 
    alpha=0.5, 
    linestyle='--'
)
ax.set_title("Monthly rate of CO2 level increase between {:s} and {:s}".format(time_min, time_max))
plt.show()

We could also use other type of geographical data provide boundaries of countries using `geopandas`.

## Working with geographic boundaries using `geopandas`

`geopandas`allows us to use other types of geographical data.
Let's use `geopandas` and read in a `geojson` file. 

In [ ]:
import geopandas as gpd
gdf_boundaries = gpd.read_file("./taking_the_pulse_atmosphere_data/world-administrative-boundaries.geojson")

In [ ]:
country_names = np.sort(gdf_boundaries.name.values)
gdf_boundaries = gdf_boundaries.set_index('name')

In [ ]:
# top10_countries

Use what is provided below in `country_names` to select the top 10 countries you found in terms of emissions. 


In [ ]:
widget_country = widgets.Select(options=country_names)
widget_country

In [ ]:
widget_country.value

Alter the name of the countries in the cell below with the top 10 countriees that you have found. 

In [ ]:

names = [
    'China', 
    'United States of America', 
    'India', 
    'Russian Federation', 
    'Japan', 
    'Iran (Islamic Republic of)', 
    'Germany',
    'Republic of Korea',
    'Saudi Arabia',
    'Indonesia'
]

In [ ]:
gdf_boundaries_top_10_co2 = gdf_boundaries.loc[names]
fig = plt.figure(figsize=(10, 5))
ax = plt.axes(projection=ccrs.PlateCarree())
gdf_boundaries_top_10_co2.plot(ax=ax, color='r')
gdf_boundaries.boundary.plot(color='k', ax=ax)

We could color code the countries with some values. Let's calculate the area of each county then plot it up. 

In [ ]:
areas = [geom.area for geom in gdf_boundaries.geometry]

In [ ]:
gdf_boundaries['area'] = areas

In [ ]:
gdf_boundaries.plot(column='area', cmap='viridis')

-----
## THE REST OF THE ASSIGNMENT - Homework Questions 

### Q1: Working with the CO2 time-series measured at Mauna Loa Observatory, calculate the mean CO2 values for each month, and plot the resulting values as a function of month; for CO2 values use the second column - "monthly_average_co2". **[Hint: use `groupby` operation]** Write the relevant codes in the below cell. A) What do you see in the plot? Describe the plot. B) Given what you have heard in class what are the possible causes of the seasonality?

In [ ]:
# Fill in this cell with codes to generate the plot

Put your answer here. 

### Q2: Working with the same time-series, calculate the trend of CO2 (i.e. rate in ppmv/decade) within each decade starting from 1950 and ending 2020 using the `linear_fit` function. Plot the resulting rates as a function of the starting year of the decade. Write the relevant codes in the cell below. Which decade shows the largest CO2 increase?

In [1]:
# Fill in this cell with codes to generate the plot

Put your answer here. 

### Q3: Working with the gridded CO2 data over the entire time span, produce the trend map. Then in the same plot show the boundaries of the 10 countries you identified as the top for emissions. Write the relevant codes in the cell below. Is there a relationship between the location of the top 10 countries and the location of regions with a high CO2 rate? Explain your observation. 

In [2]:
# Fill in this cell with codes to generate the plot

Put your answer here. 

### Q4: Choose a gridded data set from among three given below and produce a trend map, showing the coast lines on the map. 

- CH4: ./taking_the_pulse_atmosphere_data/ch4.nc
- NO2: ./taking_the_pulse_atmosphere_data/no2.nc
- CO: ./taking_the_pulse_atmosphere_data/co.nc

In [3]:
# Fill in this cell with codes to generate the plot

Put your answer here. 